# clustering
this project will be finding k optimal center points for a given data set and effectively divide the data set into clusters
## use-cases 
- labeling data under specific names
- give images a mosaic effect
- much more 

## import 

In [203]:
import numpy as np
import pandas as pd
import matplotlib as plt
import random
import json
from json import JSONEncoder

print('imports done successfully')

imports done successfully


# the point class

In [204]:
from typing_extensions import Self


class point:

    def __init__(self, position: tuple) -> None:
        self.__position = position
        self.dimension = len(position)

    @property
    def position(self) -> tuple:
        return self.__position

    def __repr__(self) -> str:
        return f'{__class__.__name__}{self.position}'

    @position.setter
    def position(self, new_position: tuple) -> None:
        assert len(
            new_position) == self.dimension, f"new position must have same dimension as old position, old dimension {self.dimension}"
        new_position = [int(i) for i in new_position]
        print(f'position changed from {self.__position} to {new_position}')
        self.__position = new_position

    def distance_squared(self, other: Self) -> float:
        assert self.dimension == other.dimension, "points of different dimensions"
        return sum([(i - j)**2 for i, j in (zip(self.position, other.position))])
    


In [188]:
# testing block

point1 = point((1,2,3,4))
print(point1)
my_tuple = 1,1,1,1.2323
point1.position = my_tuple
print(point1)
point2 = point((1,5,2,3))
print(point2)
point1.distance_squared(point2)

point(1, 2, 3, 4)
position changed from (1, 2, 3, 4) to [1, 1, 1, 1]
point[1, 1, 1, 1]
point(1, 5, 2, 3)


21

In [237]:
# read data from file
path = 'asuka.csv'
pandas_csv = pd.read_csv(path)
pandas_data = pandas_csv.iloc[:, 0: 3].copy()
reds, greens, blues = pandas_data.get(
    'r'), pandas_data.get('g'), pandas_data.get('b')
pandas_data.head()  # check
print(pandas_data)
print(pandas_csv)

          r    g    b
0       254  254  254
1       254  254  254
2       254  254  254
3       254  254  254
4       254  254  254
...     ...  ...  ...
758275   91   92  156
758276   90   92  155
758277   90   93  154
758278   89   93  152
758279   88   92  148

[758280 rows x 3 columns]
          r    g    b
0       254  254  254
1       254  254  254
2       254  254  254
3       254  254  254
4       254  254  254
...     ...  ...  ...
758275   91   92  156
758276   90   92  155
758277   90   93  154
758278   89   93  152
758279   88   92  148

[758280 rows x 3 columns]


making an array of points from cvs data

In [206]:
centers = list()
k = 3
dimension = len(pandas_data.columns)
for _ in range(k):
    cords = []
    for _2 in range(dimension):
        cords.append(random.randint(0, 255))
    centers.append(point(tuple(cords)))
center_children = {i:[] for i in centers}
print(center_children)

{point(23, 72, 244): [], point(152, 49, 2): [], point(69, 123, 114): [], point(115, 199, 124): [], point(3, 99, 70): [], point(20, 73, 16): [], point(251, 75, 4): [], point(42, 50, 45): [], point(177, 41, 151): [], point(229, 99, 164): []}


In [207]:
def winner_parent(centers: list, point: point) -> point:
    min: float = -1
    winner = None
    for center in centers:
        distance = point.distance_squared(center)
        if min == -1 or min > distance:
            min = distance
            winner = center
    return winner


In [208]:
points = []
for rgb in pandas_data.values:
    lst = []
    for i in range(dimension):
        lst.append(rgb[i])
    points.append(point(tuple(lst)))

In [209]:
for _point in points:
    center_children[winner_parent(centers,_point)].append(_point.position)

In [219]:
for center in center_children:
    if len(center_children[center]) == 0:
        continue
    a = np.average(center_children[center] , axis=0)
    center.position = a

position changed from [145, 67, 43] to [145, 67, 43]
position changed from [88, 97, 140] to [88, 97, 140]
position changed from [152, 182, 190] to [152, 182, 190]
position changed from [41, 68, 102] to [41, 68, 102]
position changed from [193, 110, 70] to [193, 110, 70]
position changed from [65, 43, 54] to [65, 43, 54]
position changed from [157, 104, 104] to [157, 104, 104]
position changed from [241, 240, 243] to [241, 240, 243]


In [220]:
for _point in points:
    center_children[winner_parent(centers,_point)].append(_point.position)
for center in center_children:
    if len(center_children[center]) == 0:
        continue
    a = np.average(center_children[center] , axis=0)
    center.position = a

c:\Users\hp\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
c:\Users\hp\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


TypeError: 'numpy.float64' object is not iterable

# write to json

In [238]:
class PointEncoder(JSONEncoder):
    def default(self, o):
        if isinstance(o, point):
            return {"coordinates": o.position, "dimension": o.dimension, o.__class__.__name__: True}
        return JSONEncoder.default


SyntaxError: invalid syntax (Temp/ipykernel_23948/3099608283.py, line 4)

In [222]:
with open("centers.json", 'w') as f:
        json.dump(centers, f, cls=PointEncoder, indent=4)